# Introduction
This notebook will model two main things, rigid-body physics, and robotic control systems, both focused around the application of a ping pong playing robot.

## Physics
Rigid-body physics: The physics will pertain to bouncing and rolling of a ping pong ball off of arbitrarily sized, positioned, and rotated, rigid body spheres, rectangular prisms, and cylinders (maybe). I wanted to make a general rigid-body simulation for completeness, but in the end all we really need is just the physics of a ball on a flat table, and an arbitrarily positioned and rotated paddle. The paddle will be made up of either two cylinders, or two rectangular prisms, which would form the handle and the paddle surface.


## Control
Robotic control systems: The first step is to simulate control over the paddle where we can magically set the paddles position and rotation or 6 degrees of freedom (DOF). The second step would be to attach the paddle to the end of a 6-DOF robot arm and control it via inverse kinematics of the robot arm, and simulating motor torques etc.

## Why we are simulating
I have the goal of getting an actual robot arm to play ping pong. The actual robot arm that I have is very expensive and I don't want to risk damaging it with potentially buggy/nonfunctional code. By simulating the ping pong physics and robot control, we can test our control code in a virtual safe space, and see how the robot arm will move and if it will take actions that could be damaging to the physical arm. When we are happy with the results of the simulated robot, we can carefully and slowly move features onto the physical robot!

Another benefit of having it in simulation, is we can change things that we couldn't change on the physical robot, like increasing the simulation speed, to run many simulations of different control gains or strategies or things like to experimentally find the optimal solution in a way that just wouldn't be plausible on the feasible robot. We could even run reinforcement learning algorithms like [Proximal Policy Optimization](https://en.wikipedia.org/wiki/Proximal_policy_optimization) on the simulated arm, which requires trying and failing at a task thousands of times.

So for these reasons of safety and convenience it is a no brainer that simulating this scenario will be greatly beneficial to the development of this robot arm application!

# Background

## Physics
At its core this notebook is a rigid-body physics simulation. The ball will be affected by gravity, and then bounce off or roll on whichever surfaces it collides with. For bouncing collisions, we will assume that balls mass to be negligible compared to the mass of the paddle/table, so the paddle/table will not budge at all under a collision.

When bouncing, the balls velocity vector will be reflected across the normal of the surface it's colliding with.

When rolling, the ball will have component forces of gravity pushing it in it's rolling state based off the normal of the surface it is rolling on. **I ended up not having time to implement rolling physics, and focused on just having the robot always be bouncing the ball off the paddle or other surfaces.**

Here are some equations that are being used for the physics:
### Numerical Equations:
For the simulation of the ball movements, we are doing simple semi-implicit numerical integration of the balls position based off its velocity and gravity. This is no different than the very first project from this class.

### Analytical Equations:
#### Trajectory equations
Given acceleration $\vec{a}$ and initial velocity and position $\vec{v_0}, \vec{r_0}$. The position at a given time is:

$\vec{r}(t) = \vec{r_0} + \vec{v_0}t + \frac{1}{2} \vec{a} t^2$

If we want to intercept the ball when it reaches a certain height $y_d$, based off its initial velocity and position $v,y$ we know that the time that it reaches that height will be:

$t_d = \frac{v \pm \sqrt{v^2-2g(y_d-y)}}{g}$ and the position and velocity at that time is simply $\vec{r}(t_d)$, $\vec{v}(t_d)$

If we want to control the ball, we need to know with what velocity to hit it to make it reach a desired position. The velocity needed for the ball to reach a desired position, given an initial position is a bit more complex. First of all, it has infinite solutions. Given any trajectory that points sufficiently up enough, there will be a velocity that will make the ball reach the target. We can format this as an optimization problem, where we want to choose a trajectory that minimizes ball velocity, but to simplify things lets just assume the trajectory is always pointing towards the target in the horizontal plane, and pointing 45 degrees upwards from that plane. Lastly we need to define a few terms:
Desired position: $\vec{r_f}$, initial position: $\vec{r_i}$, taking their difference $\vec{r_f}-\vec{r_i}$ and flattening onto the horizontal plane we will call $\vec{r_d}$
Distance: $d=|\vec{r_d}|$, Angle: $\theta$. Given all that we have

$v = \sqrt{\frac{\frac{1}{2}g\frac{d}{\cos{\theta}}^2}{d\tan{\theta}+\vec{r_{yi}}-\vec{r_{yf}}}}$

#### Collisions
Now that we know the desired initial velocity of the ball to take it from one location to another, we need to find with what speed and orientation the paddle should be in order to hit the ball with that velocity, given the balls pre-collision velocity as well. Now this problem has no analytical solution, but we can solve it numerically. If the paddle has no velocity, then its easy to know what its orientation should be. The normal of the paddle should be the vector that bisects the incoming and desired outgoing velocity vectors of the ball. But when the paddle has velocity, the proper location of this normal gets shifted. We can have an initial guess for the paddle's orientation as the bisecting normal, then we know that the correct normal will be somewhere on the line in 3d space between the initial velocity and desired outgoing velocity. If the paddle is moving towards the ball, the normal should be pointing more towards the desired outgoing velocity, and vice versa. My dirty and quick solution (because I didn't have time to fully solve) was to randomly sample points along that line in 3d space for the paddles normal, and then from there calculate the velocity. Given the paddles initial normal we can analytically find the best velocity for that normal, although it isn't a simple equation.

First off we need to know what the outgoing velocity $\vec{v_o}$ of the ball will be after a collision with the paddle with normal $\hat{n}$ and velocity $\vec{v_p}$, when the ball has incoming velocity $\vec{v_i}$

If the paddle was stationary and the collision was perfectly elastic, then we know $\frac{\hat{v_o}+\hat{v_i}}{\vec{v_o} \cdot \vec{v_i}}=\hat{n}$. We can solve this for $\vec{v_0}$ by doing a reflection. Then we can adjust it by a vector determined based off the paddle velocity and collision coefficient of restitution $\alpha$

The collision only "sees" the velocity that is projected onto the collision normal, any other velocity gets ignored. So we know that the collision velocity $v_1=\vec{v_o} \cdot \hat{n}$ and $v_2=\vec{v_p} \cdot \hat{n}$

If the ball and paddle have masses $m_1, m_2$ then the the outgoing velocity of the ball (projected on the normal) is equal to:

$v_o = (1+\alpha)\frac{m_1v_1+m_2v_2}{m_1+m_2} - \alpha v_1$

We will assume that the paddle has infinite mass relative to the ping pong ball and won't be affected by the collision which simplifies this equation to be:

$v_o = (1+\alpha)v_2 - \alpha v_1$

This magnitude will added to the velocity from before (when we assumed stationary paddle and perfect elasticity) along the direction of the normal.

Now we can find the desired paddle velocity! One important simplification is that we know the paddles velocity will be projected on the normal, so we don't need to bother giving its velocity any direction that isn't equal to the normal. We are assuming that ball spin has no affect on our physics, which allows us to make this simplification. So the desired magnitude of the velocity of the paddle (along the normal) is:

$v_p = \frac{v_o - \alpha v_1}{1+\alpha}$

So we have our guessed normal, we calculate $v_p$ as above, then we calculate the ball's trajectory based off of $v_p$ and $\hat{n}$, and see how close it is to our desired trajetory. Then we adjust the guessed $\hat{n}$ and try again, until we reach a trajectory with sufficiently low error!

#### Collision detection
As always is true with physics simulations/video games. Collision detection is an extremely complex topic that even triple A game studios can't get perfectly right. This is evidenced if you've ever watched a speedrun of Zelda: Breath of the Wild. One of the most technically advanced video games ever created in this modern era, yet the player is still easily able to break the games collision detection system and clip through walls. My point here is that I'm not going to include equations for my collision detection because it's too complex to easily represent with equations, but I'll represent it in words.

I'm only checking for collisions between the ball (a sphere) and then either sphere entities or box entities. The collision with another sphere is trivial. The collision with a box is a bit more complicated. For each side of the box, we can extend that side to be an entire plane, and we can get the distance between the sphere and that plane using some linear algebra and the plane's normal. We can then check if the sphere is touching a plane by seeing if the difference is less than its radius. If it is touching a plane, we need to make sure that it is between the four other planes that would bound that side of the box. So it's distance between each set of sides need to be greater than 0 for one side and less than 0 for the other, otherwise that would mean it is outside of the bounds.


## Control

### My initial thoughts at the start of the project
In the first step, the robot's state will be represented as the position and velocity of the ball, followed by the position and rotation of the paddle.

In the second step, the robot's state will be the same, but we will need the velocities and angular velocities of the paddle as well.

The two control modes will be bouncing and rolling, and I will likely implement some sort of sliding mode when the ball is bouncing at a very high frequency. A sliding mode is when the objects state space moves along the edge of the boundary between two different modes, known as the sliding surface.

In the bouncing mode, we can do closed form integration of the balls trajectory, given an initial velocity and position, and then determine where we want to move the paddle to, to intercept it.

In the rolling mode, we will do either PID or state-feedback control to determine how we should position and rotate the paddle to achieve the desired ball position.

### What I actually did
So because we can find the desired velocity of the paddle to hit the ball to any desired location. This actually simplifies the control a lot (and also I never implemented rolling, which the control of that is way more complicated) because all we have to do is control the paddle to the desired velocity! The physical robot arm I have has actuators with built in position control, so we don't have to do any advanced manipulator control like [resolved motion rate control](https://roboticsknowledgebase.com/wiki/planning/resolved-rates/), and we can instead just make a path of end effector positions with desired times for the manipulator to reach that position. We can then use inverse kinematics to find the joint angles needed to reach that position, and just send those angles to the actuators which have their own closed loop position control on those angles. We then step through the positions in our path at the times denoted, which will have the effect of moving the end effector (ping pong paddle) at the desired velocity!

# Approach
## Influence Diagram
Below is my influence diagram, it highlights which of the main components of the system interact with each other. "Table" can also just be any solid rectangular prism in 3d space.
![Influence Diagram](./assets/influence_diagram.png)

## Simulation Diagram
Below is my simulation diagram, it goes further into depth on how the components of the system interact, and lays a groundwork for how the math and simulation flow can be created
![Simulation Diagram](./assets/simulation_diagram.png)

## Simulation flowchart
Below is my simulation flowchart, it describes the program flow. The "collideables", "movables", and "controllables" are essentially interfaces that I defined. A collideable object must implement the `collideSphere` function and the ping pong ball will loop through all of them to check for collisions. Movables are any objects that aren't fixed in place, and controllables are any object that can control itself in some way, which for this project is just the ping pong paddle/robot arm.
![Simulation Flowchart](./assets/simulation_flowchart.png)

## Course Concepts
This project is similar to the first 3 projects from the class as it is mostly a physics simulation. Like those projects it uses continuous space and continuous time (assuming small enough dt means its continuous). Each simulation loop the time will increase by the timestep, and objects will move by an amount equal to their velocity times the timestep. For updating the objects positions and velocity, this simulation uses the semi-implicit technique, since it is the most straight-forward, and also is numerically stable. I used the same equations of "ball falling" and "ball bouncing" as the basis for my simulation. I started out my simulation process by essentially recreating project 1. The other equations I used didn't come directly from anything we learned in this class, but I used the same methods of researching the physics to find proper equations and rules to use!

# Verification Process
## Physics
### Qualitative Verification
I initially qualitatively verified my model just by watching the ball bounce around on objects, and comparing it to my mental model of how that is supposed to look. Later I further verified it by using [Physion Physics Engine](https://physion.net/) to recreate one of my scenarios and looking at the two results there and seeing that they were the same (enough). Physion is a 2d engine, so I created a 3d setup where only 1 vertical plane of the setup is relevant. In the **Testing Objects** section later, you will see something similar to the setup I used to compare to Physion, but it is a bit more complicated because I added 3d only rotation of the objects to make it more complex.

This is the setup I used in Physion:  
![My attempt at Physion](./assets/physionSim.png)

This is a similar setup in my simulation:  
![Similar to Physion setup](./assets/twoSlopesBouncing.png)  
As you can see, these setups aren't identical, but they both represent a similar enough scenario of the ball bouncing back and forth between two sloped planes, and I qualitatively verified that they both behaved in the same way.

### Quantitative verification
#### PhET
For quantitative verification I initially used the PhET onlines labs, both [PhET Collision Lab](https://phet.colorado.edu/sims/html/collision-lab/latest/collision-lab_all.html) and [PhET Projectile Motion Lab](https://phet.colorado.edu/sims/html/projectile-motion/latest/projectile-motion_all.html) to confirm that my math for my analytical projectile motion, and the collisions were correct. The projectile motion lab is only 2d, but once the ball has its initial velocity, projectile motion is always constrained to the 2d vertical plane along which the ball travels, so 3d projectile motion is inherently 2d at its core.

PhET Collisions:  
![PhET Collision Pic](./assets/phetCollisionSim.png)

PhET Trajectories:  
![PhET Trajectory Pic](./assets/phetTrajectorySim.png)

#### Self-verifying
I also somewhat self verified, as in a later feature in my simulation proved that earlier features work correctly. In the background section I discuss how I analytically solve for ball trajectories, and paddle velocities needed to hit the ball to a target position. Well in the **Testing Control** section, you'll see that when given a target position, and putting the paddle at the calculated initial velocity, it hits the ball to the correct position, meaning that the model had to have been correct!

## Control
I didn't have time to get too deep into the control of the robot, but when doing just the ball bouncing and not ball rolling the control is really quite simple. Especially because my robots actuators have internal position feedback like I mentioned earlier. The only control setup I had time to do is in the **Testing Control** section, which shows the *"robot"* can hit the ball to a target location.

In the future I could further verify the control by running the program on my actual robot arm, and see how it compares to the simulation if I used the same control gains and whatnot.

For this to work we need closed loop feedback on the position of the ball. I also have access to an XBox360 Kinect (RGBD Camera essentially) which we can use to track the location of the ball. This would obviously add noise and a time delay to the control, so I can try simulating noise and time delay as well!

# Scenarios


## Scenario 1
### Title
Two sloped boxes
### Initial conditions and parameters
The ball starts with no initial velocity, above two boxes that are both pointing at eachother around 45 degrees. One of the boxes is also slightly rotated in the third dimension, causing the ball to move towards the camera slightly when it bounces off it.
### What it represents/Why I included
This scenario represents the ball having accurate collision detection and collision physics with sloped boxes, which is a challenging and important feature of a rigid body physics simulation, and is needed for the ping pong paddle to work accurately
### Verification vs Experimental
This scenario is for qualitative verification. This scenario is slightly modified from the qualitative scenario I mentioned earlier that I compared to the Physion simulation. The only different is the slight rotation in the third dimension of the other box. So I compared it to Physion when the box didn't have the slight rotation, then I added the slight rotation and I compared it to my mental model of how it should move

## Scenario 2
### Title
Ping Pong table between two walls
### Initial conditions and parameters
The ball starts with a high initial velocity in the x direction, and slight velocity in the z direction, sitting above an accurately sized ping pong table which is sitting between two large walls. The ball bounces up and around back and forth on the table and walls, eventually hitting the net, and losing enough vertical velocity that it "rolls" along the table while bouncing back and forth between the net and wall. I say "rolls" because there is no actual rolling physics, it just is being held up by the table and sliding back and forth.
### What it represents/Why I included
I included this to see what would happen when I push the edgecases of the ball going into a "rolling" state, and colliding with multiple objects at once. It's also testing the ping pong table, which is a hybrid object made up of a net and a table. This is useful because the ping pong paddle is also a hybrid object made up of handle and paddle.
### Verification vs Experimental
This scenario is more experimental, but you can also observe the result as a qualitative verification

## Scenario 3
### Title
Control paddle to hit ball to destination
### Initial conditions and parameters
The ball starts with no initial velocity, sitting above a ping pong paddle with a random upwards pointing normal. There is also a yellow sphere with a random position, this is the destination. The ball falls onto this paddle and then gains a random trajectory. Then the paddle control kicks in, causing it to move in a way that intercepts the ball when it reaches the initial height of the bounce, and collides with the ball in a way that gives it the velocity needed to travel through the yellow sphere
### What it represents/Why I included
I included this to test that the paddle velocity control is correct. This is crucially important for getting the robot to control the ball
### Verification vs Experimental
This scenario was for quantiative self-verification. As I mentioned in the **Verification Process** section, for the ball to reach the destination, everything else about my model *must* be correct. I already quantiatively verified my collision and trajectory equations using PhET, so the only variable still is the model/numerical integration simulation. So because the velocity of the paddle is being calculated using verified analytical equations, if the ball reaches the destination, then the model/simulation *must* be accurate. And as you'll see, it is!

## Scenario 4
### Title
Rendering robot arm from Universal Robot Description Format file (URDF) and controlling it with inverse kinematics
### Initial conditions and parameters
We render a robot arm, and give it a desired path to follow, which is just a straight line across the range of places it can reach. And also a target orientation of the end effector, which is just pointing up. On each time step we calculate the desired joint angles using IK, and then render the arm with those joint angles. And also render the goal end effector position as a green sphere.
### What it represents/Why I included
I included this to test the control of the robot arm, and as a pre-requisite for inserting the robot arm into the main simulation, and attaching the ping pong paddle to it etc.
### Verification vs Experimental
This scenario is more experimental, as I haven't had time to get too deep into including the model of the robot arm in the simulation. But its also been verified, as the URDF file I used is based off of the actual robot arm that I have. And I ran the IK both in the simulation and on the real arm, and the real arm was able to do the path tracking that you'll see in the simulation. I should've gotten a video of the arm doing this but I forgot and don't have time to get one now. In the coming week if I get the arm working more, I might post an update that includes the physical arm working.

# Results (Code for now)

## Imports and Globals
Here we do necessary imports and also define some global variables/settings/parameters
`floorActive` determines if the ball will bounce up when it reaches y=0  
`preSimulateThenPlayback` if true, will not render the scene at all, and only compute the physics and control in the backend, while saving the moveable objects position over time. Then when that has completed, it will playback the scene, to have a smoother viewing experience. This hasn't been tested recently so might not work

In [1]:
from vpython import *
import numpy as np
import time
G = -9.8
floorActive = False
fps = 60
speedMultiplier = 1
dt = 0.0005 * speedMultiplier
resolutionMultiplier = int(1.0/(dt*fps)*speedMultiplier)
print(resolutionMultiplier)
preSimulateThenPlayback = False
scene = canvas()
sphere()

<IPython.core.display.Javascript object>

33


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Function Definitions

In [2]:
# Returns a rotation matrix R that will rotate vector a to point in the same direction as vector b
def alignVectorsRotationMatrix(a, b):
    b = b / np.linalg.norm(b) # normalize a
    a = a / np.linalg.norm(a) # normalize b
    v = np.cross(a, b)
    # s = np.linalg.norm(v)
    c = np.dot(a, b)
    if np.isclose(c, -1.0):
        return -np.eye(3, dtype=np.float64)

    v1, v2, v3 = v
    h = 1 / (1 + c)

    Vmat = np.array([[0, -v3, v2],
                  [v3, 0, -v1],
                  [-v2, v1, 0]])

    R = np.eye(3, dtype=np.float64) + Vmat + (Vmat.dot(Vmat) * h)
    return R
def axisAngleRotationMatrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    a = cos(theta / 2.0)
    b, c, d = -axis * sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
def yawRotationMatrix(theta):
    return np.array([[cos(theta), 0, sin(theta)],
                     [0, 1, 0],
                     [-sin(theta), 0, cos(theta)]])
    
def pitchRotationMatrix(theta):
    return np.array([[cos(theta), -sin(theta), 0],
                     [sin(theta), cos(theta), 0],
                     [0, 0, 1]])
def rollRotationMatrix(theta):
    return np.array([[1, 0, 0],
                     [0, cos(theta), -sin(theta)],
                     [0, sin(theta), cos(theta)]])

# if m2 is negative, it assumes m2 has infinite mass
# so (m1*v1+m2*v2)/(m1+m2) = m2*v2/m2 = v2 as m2 approaches infinity
def collision(m1, v1, m2, v2, alpha):
    vp = (1+alpha)*((m1*v1+m2*v2)/(m1+m2) if m2 > 0 else v2)
    v1p = vp - alpha*v1
    v2p = vp - alpha*v2
    ## These were for testing
    # kei = 0.5*m1*v1**2 + 0.5*m2*v2**2
    # kef = 0.5*m1*v1p**2 + 0.5*m2*v2p**2
    # foundAlpha = kef / kei
    return v1p, v2p#, kei, kef, foundAlpha

## Class Definitions

In [3]:
# Abstract
class Object:
    def __init__(self, pos=np.zeros(3)):
        self.pos = np.array(pos, dtype=float)
        self.vel = np.zeros(3)
    
    def show(self):
        self.visual.pos.x = self.pos[0]
        self.visual.pos.y = self.pos[1]
        self.visual.pos.z = self.pos[2]
        
class Sphere(Object):
    def __init__(self, pos=np.zeros(3), radius=1, color=vec(0.4, 0.4, 0.4), make_trail=False):
        super().__init__(pos=pos)
        self.radius = radius
        self.visual = sphere(make_trail = make_trail, pos=vec(0,0,0), radius=radius, color=color)
        self.show()
        
    def move(self, pos=None):
        if np.all(pos != None):
            self.pos = pos
        
    def collideSphere(self, other):
        displacement = other.pos - self.pos
        if np.linalg.norm(displacement) < self.radius + other.radius:
            collisionNormal = displacement/np.linalg.norm(displacement)
            return collisionNormal, collisionNormal*self.radius
        return None, None
        
class Box(Object):
    def __init__(self, pos=np.zeros(3), size=np.ones(3), axis=[1,0,0], angle=0, color=vec(0.4, 0.4, 0.4), showNormals=False):
        super().__init__(pos=pos)
        self.size = np.array(size, dtype=float)
        self.axis = np.array(axis, dtype=float)
        self.angle = 0
        self.axisRotation = None
        self.angleRotation = None
        self.visual = box() # make default box for now, it will get redefined in the move
        self.visual.color = color
        self.showNormals = showNormals
        self.arrows = []
        self.move(axis=axis, angle=angle, showNormals=showNormals)
        self.show()
        
    def move(self, pos=None, axis=None, upAxis=None, angle=None, showNormals=None, axisRotation=None, angleRotation=None):
        prevAxis = self.axis
        prevAngle = self.angle
        useUpAxis = False
        if np.all(pos != None):
            self.pos = pos
        if np.all(axis != None):
            self.axis = np.array(axis, dtype=float)
            self.axis /= np.linalg.norm(self.axis)
        elif np.all(upAxis != None):
            useUpAxis = True
        if angle != None:
            self.angle = angle
        if showNormals != None:
            self.showNormals = showNormals
        
        if np.any(self.axis != prevAxis) or np.any(self.angle != prevAngle) or np.all(self.axisRotation == None) or useUpAxis:
            if np.all(axisRotation != None) and np.all(angleRotation != None):
                self.axisRotation = axisRotation
                self.angleRotation = angleRotation
            elif useUpAxis:
                self.axisRotation = alignVectorsRotationMatrix(np.array([0,1,0]), upAxis)
                self.angleRotation = axisAngleRotationMatrix(upAxis, self.angle)
            else:
                self.axisRotation = alignVectorsRotationMatrix(np.array([1,0,0]), self.axis)
                self.angleRotation = axisAngleRotationMatrix(self.axis, self.angle)
            # maybe rework to adjust existing np array instead of creating new one each move
            self.normals = np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]], dtype=float)
            for i in range(6):
                self.normals[i] = np.dot(self.axisRotation, self.normals[i])
                self.normals[i] = np.dot(self.angleRotation, self.normals[i])
            if useUpAxis:
                self.axis = self.normals[0]

        
        
        
    def collideSphere(self, other):
        # the dot product is a neat way of finding minimum distance between point and plane
        minDistances = [np.abs(np.dot(other.pos - (self.pos+self.size[int(i/2)]*self.normals[i]*(0 if i%2==1 else 1)), self.normals[i])) for i in range(6)]
        for i in range(6):
            if minDistances[i] < other.radius:
                wallsToCheck = [j for j in range(6) if j != i and j != i+(1 if i%2==0 else -1)]
                outsideWalls = False
                for j in wallsToCheck:
                    if minDistances[j] > self.size[int(j/2)]:
                        # print(str(j) + " | " + str(minDistances))
                        outsideWalls = True
                if not outsideWalls:
                    collisionPoint = other.pos - self.normals[i]*minDistances[i]
                    return self.normals[i], collisionPoint
        return None, None
        
    def update(self):
        pass
            
    def show(self):
        
        self.visual.axis.x = self.axis[0]
        self.visual.axis.y = self.axis[1]
        self.visual.axis.z = self.axis[2]
        
        self.visual.up.x = self.normals[2][0]
        self.visual.up.y = self.normals[2][1]
        self.visual.up.z = self.normals[2][2]
        
        self.visual.size.x = self.size[0]
        self.visual.size.y = self.size[1]
        self.visual.size.z = self.size[2]
        
        # alignedDiagonal = self.size
        alignedDiagonal = np.dot(self.axisRotation, self.size)
        alignedDiagonal = np.dot(self.angleRotation, alignedDiagonal)
        self.pos += alignedDiagonal/2
        super().show()
        if self.showNormals:
            if len(self.arrows)==0:
                for normal in self.normals:
                    shortenedNormal = normal/(np.linalg.norm(normal)*4)
                    self.arrows.append(arrow(pos=vec(self.pos[0],self.pos[1],self.pos[2]), axis=vec(shortenedNormal[0],shortenedNormal[1],shortenedNormal[2])))
            else:
                for i in range(len(self.arrows)):
                    shortenedNormal = self.normals[i]/(np.linalg.norm(self.normals[i])*4)
                    self.arrows[i].axis = vec(shortenedNormal[0],shortenedNormal[1],shortenedNormal[2])
                    self.arrows[i].pos = vec(self.pos[0],self.pos[1],self.pos[2])
                    
        self.pos -= alignedDiagonal/2
        
        
class Ball(Sphere):
    def __init__(self, pos=np.zeros(3), make_trail=False):
        super().__init__(pos=pos, radius=0.02, color=vec(1,1,1), make_trail=make_trail)
        self.restitution = 0.8
        self.mass = 0.0027 # This value is currently irrelevant, because we are assuming all other objects have infinite mass
        
    def update(self, collideables=[]):
        self.vel += np.array([0,1,0])*G*dt
        self.pos += self.vel*dt
        
        
        
        for obj in collideables:
            collisionNormal, collisionPoint = obj.collideSphere(self)
            if type(collisionNormal)!=list:
                collisionNormal = [collisionNormal]
                collisionPoint = [collisionPoint]
            for i in range(len(collisionNormal)):
                if np.all(collisionNormal[i] != None):
                    # First get the new velocity if the collision was perfectly elastic
                    rotMat = alignVectorsRotationMatrix(self.vel*np.array([-1,-1,-1]), collisionNormal[i])
                    self.vel = np.linalg.norm(self.vel)*np.dot(rotMat, collisionNormal[i])
                    
                    # Then find how to adjust it based off of inelasticity
                    # We do this by looking at the magnitude of the velocity projected onto the collision normal
                    projectedVelocityMag = np.dot(self.vel, collisionNormal[i])
                    # And then do a 1d collision equation between both objects projected velocities
                    # Which gives us a new velocity in this 1 dimension.
                    if hasattr(obj, "vel"):
                        otherVel = np.dot(obj.vel, collisionNormal[i])
                    else:
                        otherVel = 0 # Stationary
                    # angular vel in collisions WIP
                    if hasattr(obj, "angularVel"):
                        if np.linalg.norm(obj.angularVel) != 0:
                            # we need to find the instantaneous velocity of the collision point
                            # which is equal to the cross product of the vector from object origin to collision point
                            # and the angular velocity of the object
                            instantaneousRotatingVelocity = np.cross(obj.angularVel, collisionPoint[i])

                            # lastly we project the instantenousRotatingVelocity onto the collision normal
                            rotatingCollisionVelocity = np.dot(instantaneousRotatingVelocity, collisionNormal[i])
                            # and superimpose it with the translational velocity
                            otherVel += rotatingCollisionVelocity
                    # we use negative projectedVelocityMag so the collision function thinks they are colliding
                    newProjectedVelocityMag, _ = collision(self.mass, -projectedVelocityMag, -1, otherVel, self.restitution)
                    # print(f"NEWPROJECTVELMAG {newProjectedVelocityMag}")
                    # We take the difference in magnitude from before and after the collision
                    collisionAdjustMag = projectedVelocityMag - newProjectedVelocityMag
                    # Then add the difference to the new velocity along the direction of the collision normal
                    collisionAdjust = collisionAdjustMag * collisionNormal[i]
                    if np.dot(self.vel, collisionNormal[i]) > 0:
                        self.vel -= collisionAdjust
                    else:
                        self.vel += collisionAdjust
                    # print(f"POST COLLISION VEL {self.vel} and UNIT {self.vel/np.linalg.norm(self.vel)}")
                        
                    
                    # Adjust position to snap to collision point
                    self.pos = collisionPoint[i] + self.radius*collisionNormal[i]
        if self.pos[1] < 0 and floorActive:
            self.pos[1] = 0
            self.vel[1] *= -self.restitution
    def findInterception(self, desiredInterceptionHeight):
        timeOfInterception = (self.vel[1] + sqrt(self.vel[1]**2 + 2*G*(desiredInterceptionHeight - self.pos[1]))) / -G
        velocityOfInterception = np.array([self.vel[0], self.vel[1] + G*timeOfInterception, self.vel[2]])
        positionOfInterception = np.array([self.pos[0] + self.vel[0]*timeOfInterception, self.pos[1] + self.vel[1]*timeOfInterception + 0.5*G*timeOfInterception**2, self.pos[2] + self.vel[2]*timeOfInterception])
        return timeOfInterception, positionOfInterception, velocityOfInterception
    
    def findVelocityToReachPosition(self, desiredPosition, startingPosition=None):
        if np.all(startingPosition != None):
            pos = startingPosition
        else:
            pos = self.pos
        distanceXZ = (desiredPosition - pos)*np.array([1, 0, 1])
        
        distanceMag = np.linalg.norm(distanceXZ)
        trajectory = np.array([distanceXZ[0], distanceMag, distanceXZ[2]])
        theta = acos(np.dot(distanceXZ, trajectory)/(np.linalg.norm(trajectory)*np.linalg.norm(distanceXZ)))
        initialMagnitude = sqrt(abs(0.5*G*(np.linalg.norm(distanceXZ)/cos(theta))**2/(np.linalg.norm(distanceXZ)*tan(theta)+pos[1]-desiredPosition[1])))
        return trajectory/np.linalg.norm(trajectory)*initialMagnitude
                
        
class Table(Box):
    def __init__(self, pos=np.zeros(3)):
        super().__init__(pos=pos, size=[2.743, 0.752, 1.524], color=vec(0.3, 0.3, 0.6))

class Net(Box):
    def __init__(self, pos=np.zeros(3)):
        super().__init__(pos=pos, size=[0.005, 0.1525, 1.83], color=vec(0.7, 0.7, 0.7))

class PingPongTable():
    def __init__(self, pos=np.zeros(3)):
        self.table = Table(pos=pos)
        self.net = Net(pos=[pos[0]+self.table.size[0]/2,pos[1]+self.table.size[1],pos[2]-0.15])
    def collideSphere(self, other):
        tableNormal, tablePoint = self.table.collideSphere(other)
        netNormal, netPoint = self.net.collideSphere(other)
        return [tableNormal, netNormal], [tablePoint, netPoint]
    def show(self):
        self.table.show()
        self.net.show()
        
class PingPongPaddle():
    def __init__(self, pos=np.zeros(3)):
        
        # Object stuff
        handleSize=[0.1, 0.02, 0.03]
        paddleSize=[0.2, 0.02, 0.15]
        # self.handle = Box(pos=[pos[0], pos[1]-handleSize[1]/2, pos[2]-handleSize[2]/2], size=handleSize, showNormals=False)
        # self.paddle = Box(pos=[pos[0]+self.handle.size[0], pos[1]-paddleSize[1]/2, pos[2]-paddleSize[2]/2], size=paddleSize, showNormals=False)
        self.handle = Box(pos=[pos[0], pos[1], pos[2]], size=handleSize, showNormals=False)
        self.paddle = Box(pos=[pos[0]+self.handle.size[0], pos[1], pos[2]-paddleSize[2]/2+self.handle.size[2]/2], size=paddleSize, showNormals=True)
        
        # Physics stuff
        self.pos = np.array(pos, dtype=float)
        self.axis = self.handle.axis
        self.angle = self.handle.angle
        self.vel = np.zeros(3)
        self.angularVel = np.zeros(3)
        self.metaVel = np.zeros(3)
        self.axisRotation = None
        self.angleRotation = None
        
        # Controls stuff
        self.hitBall = False
        self.ballHit = None
        self.disableVelocity = False
        self.targetPosition = (np.random.rand(3)*2-1)*7
        if self.targetPosition[1] < 0 and floorActive:
            self.targetPosition[1] *= -1
        sphere(pos=vec(self.targetPosition[0], self.targetPosition[1], self.targetPosition[2]), radius=0.05, color=vec(1, 0.8, 0))
    
    # deprecated
    # def update(self):
    #     self.pos += self.vel*dt
    #     self.rotation += self.angularVel*dt
    #     self.axis = np.dot(yawRotationMatrix(self.rotation[0]), np.dot(pitchRotationMatrix(self.rotation[1]), np.array([1,0,0])))
    #     self.angle += self.angularVel[2]*dt
    #     self.move()
    def update(self):
        self.vel = self.vel*0
        self.angularVel = self.angularVel*0
        
        if self.hitBall and self.ballHit:
            if self.ballHit.vel[1] > 0.8:
                print("CONTROL")
                t, pos, incomingVel = self.ballHit.findInterception(self.ballHit.pos[1])
                outgoingVel = self.ballHit.findVelocityToReachPosition(self.targetPosition, startingPosition=pos)
                print(f"TARGETVEL {outgoingVel} and UNIT {outgoingVel/np.linalg.norm(outgoingVel)}")
                
                #### Make ball go to target position
                # arrow(pos=vec(pos[0], pos[1], pos[2]), axis=vec(incomingVelUnit[0], incomingVelUnit[1], incomingVelUnit[2]), color=vec(1, 0, 0))
                # arrow(pos=vec(pos[0], pos[1], pos[2]), axis=vec(outgoingVelUnit[0], outgoingVelUnit[1], outgoingVelUnit[2]), color=vec(0, 1, 0))
                
                print("ATTEMPTING NUMERICAL SOLUTION")
                ###### Guess a normal (upAxis), then find paddleMagnitude
                incomingVelUnit = -1*incomingVel / np.linalg.norm(incomingVel)
                outgoingVelUnit = outgoingVel / np.linalg.norm(outgoingVel)
                error = 99999
                threshold = 0.01
                attempts = 0
                maxAttempts = 200
                lowestError = 99999
                bestPaddleVel = 0
                prevPaddleVel = 0
                prevPrevPaddleVel = 0
                bestNormal = np.array([0, 1, 0])
                moreOutgoingNormal = outgoingVelUnit
                moreCurrentNormal = incomingVelUnit
                prevNormal = bestNormal.copy()
                
                # if True:
                while error > threshold and attempts < maxAttempts:
                    # for starting guess, assume paddle is stationary and use that correct normal
                    if attempts == 0 or True:
                        upAxis = outgoingVelUnit + incomingVelUnit # bisect target and current velocity
                        upAxis /= np.linalg.norm(upAxis)
                    # for future guesses, binary search over angle range between current and target velocity
                    # if the paddle velocity was greater than previous attempt, move normal towards target half way
                    # if paddle velocity was less than previous attempt, move normal towards last normal half way
                    # else:
                    #     if prevPaddleVel > prevPrevPaddleVel:
                    #         print("GOING OUTGOING")
                    #         upAxis = moreOutgoingNormal + prevNormal
                    #         upAxis /= np.linalg.norm(upAxis)
                    #         moreCurrentNormal = prevNormal.copy()
                    #     else:
                    #         print("GOING CURRENT")
                    #         upAxis = moreCurrentNormal + prevNormal
                    #         upAxis /= np.linalg.norm(upAxis)
                    #         moreOutgoingNormal = prevNormal.copy()
                    #     prevNormal = upAxis.copy()
                    
                    ## randomly choose normal along line of rotation between incoming and outgoing vels
                    ## slightly less dirty than below
                    upAxis = np.random.rand(1)[0]*incomingVelUnit + 0.5*outgoingVelUnit
                    upAxis /= np.linalg.norm(upAxis)
                    
                    # # randomly disturb the normal, then renormalize, use this as our normal guess
                    # ### this is super dirty and we should try to find a better way
                    # upAxis += np.random.rand(3)-0.5
                    # upAxis /= np.linalg.norm(upAxis)

                    initialMagnitude = np.linalg.norm(np.dot(outgoingVel, upAxis))
                    incomingMagnitude = np.linalg.norm(np.dot(incomingVel, upAxis))
                    # print(f"INITMAG {initialMagnitude}")
                    # comes from collision equation see below
                    paddleMagnitude = (initialMagnitude - self.ballHit.restitution*incomingMagnitude)/(1+self.ballHit.restitution)
                    
                    ###### then based on that, see what the output would be, (this code is copied from ball update, check there for comments/more descriptions)
                    rotMat = alignVectorsRotationMatrix(incomingVel*np.array([-1,-1,-1]), upAxis)
                    reflectedVel = np.linalg.norm(incomingVel)*np.dot(rotMat, upAxis)
                    projectedVelocityMag = np.dot(reflectedVel, upAxis)
                    newProjectedVelocityMag, _ = collision(self.ballHit.mass, -projectedVelocityMag, -1, paddleMagnitude, self.ballHit.restitution)
                    
                    collisionAdjustMag = projectedVelocityMag - newProjectedVelocityMag
                    
                    collisionAdjust = collisionAdjustMag * upAxis
                    if np.dot(reflectedVel, upAxis) > 0:
                        finalVel = reflectedVel - collisionAdjust
                    else:
                        finalVel = reflectedVel + collisionAdjust
                    # print(f"PREDICTED VEL {finalVel}")
                    error = np.linalg.norm(outgoingVel - finalVel)
                    # print(f"ERROR {error}")
                    
                    # Save the best result to use later
                    if error < lowestError:
                        # print("NEW BEST SOLUTION FOUND ON ATTEMPT", attempts)
                        prevPrevPaddleVel = prevPaddleVel
                        prevPaddleVel = paddleMagnitude
                        bestPaddleVel = paddleMagnitude
                        bestNormal = upAxis.copy()
                        lowestError = error
                        
                    attempts += 1
                    
                
                if attempts == maxAttempts:
                    print("MAX ATTEMPTS OF", maxAttempts, "REACHED")
                    print("BEST SOLUTION HAS ERROR", lowestError)
                else:
                    print("NUMERICAL SOLUTION TOOK", attempts, "ATTEMPTS")
                    print("FINAL SOLUTION HAS ERROR", lowestError)
                
                upAxis = bestNormal
                paddleMagnitude = bestPaddleVel
                self.metaVel = upAxis * paddleMagnitude
                self.move(pos=pos-self.metaVel*t, upAxis=upAxis, angle=0, alignCenter=True)
                
                ####### make ball return to where it came from
#                 t, pos, vel = self.ballHit.findInterception(self.ballHit.pos[1])
#                 upAxis = -1*vel
#                 defaultUpAxis = np.array([0, 1, 0])
#                 rotMat = alignVectorsRotationMatrix(defaultUpAxis, upAxis)
#                 defaultAxis = np.array([1,0,0])
#                 defaultAxis = np.dot(rotMat, defaultAxis)
                
#                 distance = (pos - self.ballHit.pos)*np.array([1, 0, 1])
                
#                 theta = acos(np.dot(distance, upAxis)/(np.linalg.norm(upAxis)*np.linalg.norm(distance)))
#                 initialMagnitude = sqrt(abs(0.5*G*(np.linalg.norm(distance)/cos(theta))**2/(np.linalg.norm(distance)*tan(theta)+self.ballHit.pos[1]-pos[1])))
                
#                 # we want ball to leave collision with velocity magnitude of initialMagnitude
#                 # it is coming in with self.ballHit.vel velocity
#                 # we need to find metaVel which would be v2 to make the collision output initialMagnitude
#                 # vp is an intermediate, v1 is ball vel, v2 is paddle vel. v1p is output vel
#                 # vp = (1+alpha)*((m1*v1+m2*v2)/(m1+m2) if m2 > 0 else v2)
#                 # above simplifies to vp = (1+alpha)*v2 because of infinite mass paddle
#                 # v1p = vp - alpha*v1
#                 # find v2 such that v1p = initialMagnitude
#                 # initialMagnitude = vp - alpha*v1 = (1+alpha)*v2 - alpha*v1
#                 # v2 = (initialMagnitude + alpha*v1)/(1+alpha)
                
#                 paddleMagnitude = (initialMagnitude - self.ballHit.restitution*np.linalg.norm(vel))/(1+self.ballHit.restitution)
                
#                 self.metaVel = upAxis/np.linalg.norm(upAxis) * paddleMagnitude
                
                
#                 self.move(pos=pos-self.metaVel*t, axis=defaultAxis, angle=0, alignCenter=True)
                
                
                
                ###### Find the paddle velocity needed to keep the ball at the same height
                # t, pos, vel = self.ballHit.findInterception(self.ballHit.pos[1])
                # ball is coming in with velocity v1, if we have velocity v2 then:
                # vp = (1+alpha)*((m1*v1+m2*v2)/(m1+m2) if m2 > 0 else v2)
                # v1p = vp - alpha*v1
                # where vp is an intermediate, and v1p is the output velocity
                # we want to find v2 such that v1p is equal to -v1
                # but we know that m2 will be considered infinite, so the vp expression simplifies to:
                # vp = (1+alpha)*v2
                # which gives: v1p = (1+alpha)*v2 - alpha*v1
                # solving for v2 to find v1p=-v1 gives: v2=(alpha*v1-v1)/(1+alpha)=v1*(alpha-1)/(1+alpha)
                # self.metaVel[1] = self.ballHit.vel[1]*(1-self.ballHit.restitution)/(1+self.ballHit.restitution)
                # this code puts the ball straight back up at same velocity
                # self.move(pos=pos-self.metaVel*t, axis = [1,0,0], alignCenter=True)
                # # self.move(pos=pos, axis = [1,0,0], alignCenter=True)
                
            self.hitBall = False
            
        if np.any(self.metaVel != 0):
            self.move(pos=self.pos+self.metaVel*dt, alignCenter=False)
        
    def move(self, pos=None, axis=None, upAxis=None, angle=None, alignCenter=False):
        prevPos = self.pos
        prevAxis = self.axis
        prevAngle = self.angle
        prevUpVector = self.handle.normals[2]
        useUpAxis=False
        if np.all(axis != None):
            self.axis = np.array(axis, dtype=float)
        elif np.all(upAxis != None):
            useUpAxis=True
        if angle != None:
            self.angle = angle
        
        if useUpAxis:
            if np.any(prevUpVector != upAxis):
                self.axisRotation = alignVectorsRotationMatrix(np.array([0,1,0]), upAxis)
                self.angleRotation = axisAngleRotationMatrix(upAxis, self.angle)
        elif np.any(self.axis != prevAxis) or np.any(self.angle != prevAngle) or np.all(self.axisRotation == None):
            self.axisRotation = alignVectorsRotationMatrix(np.array([1,0,0]), self.axis)
            self.angleRotation = axisAngleRotationMatrix(self.axis, self.angle)
            
        if np.all(pos != None):    
            self.pos = np.array(pos, dtype=float)
        if alignCenter:
            centerAdjust = np.array([self.handle.size[0]+self.paddle.size[0]/2, 0, 0]) # bottom back left corner aligned coordinates
            # account for rotation
            centerAdjust = np.dot(self.axisRotation, centerAdjust)
            centerAdjust = np.dot(self.angleRotation, centerAdjust)
            self.pos = self.pos-centerAdjust
        self.vel = (self.pos - prevPos)/dt
        if useUpAxis:
            self.handle.move(pos=self.pos, upAxis=upAxis, angle=self.angle, axisRotation=self.axisRotation, angleRotation=self.angleRotation)
            self.axis = self.handle.normals[0]
        else:
            self.handle.move(pos=self.pos, axis=self.axis, angle=self.angle, axisRotation=self.axisRotation, angleRotation=self.angleRotation)
        
        paddleAdjust = np.array([self.handle.size[0], 0, self.handle.size[2]/2-self.paddle.size[2]/2]) # bottom back left corner aligned coordinates
        # paddleAdjust = np.array([self.handle.size[0], -self.paddle.size[1]/2, -self.paddle.size[2]/2]) # principle axis aligned coordinates
        # paddleAdjust = np.array([self.handle.size[0]/2+self.paddle.size[0]/2, 0, 0]) # center points aligned coordinates
        
        # account for rotation when placing paddle relative to handle
        paddleAdjust = np.dot(self.axisRotation, paddleAdjust)
        paddleAdjust = np.dot(self.angleRotation, paddleAdjust)
        if useUpAxis:
            self.paddle.move(pos=self.pos+paddleAdjust, upAxis=upAxis, angle=self.angle, axisRotation=self.axisRotation, angleRotation=self.angleRotation)
        else:
            self.paddle.move(pos=self.pos+paddleAdjust, axis=self.axis, angle=self.angle, axisRotation=self.axisRotation, angleRotation=self.angleRotation)
        
        if np.any(self.axis != prevAxis) or np.any(self.angle != prevAngle):
            # Find angular velocity based on change in axis and up vector
            dAxis = (self.axis - prevAxis) / dt
            dUp = (self.handle.normals[2] - prevUpVector) / dt
            # angularVel = (x X xdot)/(x . x) + x*[(y X ydot)/(y . y) - (x X xdot)/(x . x)]_n / (x-y)_n
            unconstrainedVector = np.cross(prevAxis, dAxis)/np.dot(prevAxis, prevAxis)
            constraintVector = np.cross(prevUpVector, dUp)/np.dot(prevUpVector, prevUpVector) - unconstrainedVector
            constraintScalar = constraintVector[0] / (prevAxis - prevUpVector)[0]
            self.angularVel = unconstrainedVector + prevAxis*constraintScalar
        
        if self.disableVelocity:
            self.vel = np.zeros(3)
            self.angularVel = np.zeros(3)
        
        
        
    def collideSphere(self, other):
        handleNormal, handlePoint = self.handle.collideSphere(other)
        paddleNormal, paddlePoint = self.paddle.collideSphere(other)
        if np.all(handleNormal != None) or np.all(paddleNormal != None) and self.ballHit == None:
            self.hitBall = True
            self.ballHit = other
        return [handleNormal, paddleNormal], [handlePoint, paddlePoint]
    def show(self):
        self.handle.show()
        self.paddle.show()

## Testing

### Testing Objects
#### Scenario 1

In [5]:
scene = canvas()
scene.autoscale = False
arrow(pos=vec(0,0,0), axis=vec(1,0,0), color=vec(1,0,0))
arrow(pos=vec(0,0,0), axis=vec(0,1,0), color=vec(0,1,0))
arrow(pos=vec(0,0,0), axis=vec(0,0,1), color=vec(0,0,1))

# give ball height 5 for test scenarios
ball = Ball(pos = [0.2, 5, 0.4], make_trail=True)
moveables = [ball]
collideables = []

### Test physics configurations:
box1 = Box(pos = [0,0,0], size=[4, 0.4, 4], axis=[-1,1,0], angle=0, showNormals=True)
box2 = Box(pos = [0,0,0], size=[4, 0.4, 4], axis=[1,1,0*-0.05], angle=2*3.14159/180, showNormals=False, color=vec(0.6, 0.4, 0.4))
collideables.append(box1)
collideables.append(box2)

moveablesStates = []
for _ in moveables:
    moveablesStates.append([])
t = 0
frames = 0
simLengthSeconds = 5
simLengthTicks = simLengthSeconds/speedMultiplier
startTime = time.time()
while t < simLengthTicks:
    for _ in range(resolutionMultiplier):
        ball.update(collideables=collideables)
        t += dt
        frames += 1
    if preSimulateThenPlayback:
        for i in range(len(moveables)):
            moveablesStates[i].append(np.copy(moveables[i].pos))
    if not preSimulateThenPlayback:
        for moveable in moveables:
            moveable.show()
        rate(fps)
endTime = time.time()

if preSimulateThenPlayback:
    print(f"Simulation took: {endTime - startTime} seconds")
    for t in range(len(moveablesStates[0])):
        for i in range(len(moveablesStates)):
            moveables[i].move(pos=moveablesStates[i][t])
            moveables[i].show()
        rate(fps)
else:
    print(f"Simulation was expected to take: {t} seconds and it took: {endTime - startTime} seconds")

<IPython.core.display.Javascript object>

Simulation was expected to take: 5.01599999999994 seconds and it took: 6.910336017608643 seconds


#### Scenario 2

In [6]:
scene = canvas()
scene.autoscale = False
arrow(pos=vec(0,0,0), axis=vec(1,0,0), color=vec(1,0,0))
arrow(pos=vec(0,0,0), axis=vec(0,1,0), color=vec(0,1,0))
arrow(pos=vec(0,0,0), axis=vec(0,0,1), color=vec(0,0,1))

# give ball height 5 for test scenarios
ball = Ball(pos = [0.2, 5, 0.4], make_trail=True)
moveables = [ball]
collideables = []

ball.vel[0]=5.0
ball.vel[2]=0.07
table = PingPongTable()
collideables.append(table)
collideables.append(Box(pos=[-1,0,0], size=[1,6,3]))
collideables.append(Box(pos=[table.table.size[0],0,0], size=[1,6,3]))

moveablesStates = []
for _ in moveables:
    moveablesStates.append([])
t = 0
frames = 0
simLengthSeconds = 10
simLengthTicks = simLengthSeconds/speedMultiplier
startTime = time.time()
while t < simLengthTicks:
    for _ in range(resolutionMultiplier):
        ball.update(collideables=collideables)
        t += dt
        frames += 1
    if preSimulateThenPlayback:
        for i in range(len(moveables)):
            moveablesStates[i].append(np.copy(moveables[i].pos))
    if not preSimulateThenPlayback:
        for moveable in moveables:
            moveable.show()
        rate(fps)
endTime = time.time()

if preSimulateThenPlayback:
    print(f"Simulation took: {endTime - startTime} seconds")
    for t in range(len(moveablesStates[0])):
        for i in range(len(moveablesStates)):
            moveables[i].move(pos=moveablesStates[i][t])
            moveables[i].show()
        rate(fps)
else:
    print(f"Simulation was expected to take: {t} seconds and it took: {endTime - startTime} seconds")

<IPython.core.display.Javascript object>

Simulation was expected to take: 10.015500000000749 seconds and it took: 27.342137575149536 seconds


### Testing Control
#### Scenario 3
This scenario can be a little wonky, beacuse of the randomness that is used for variety. It's possible we get a random configuration where the ball doesn't bounce up off the paddle initially, or there is no solution for reaching the target destination. If this happens and the ball doesn't reach the destination, just re-run the scenario. It should work most of the time.

In [9]:
scene = canvas()
scene.autoscale = False
arrow(pos=vec(0,0,0), axis=vec(1,0,0), color=vec(1,0,0))
arrow(pos=vec(0,0,0), axis=vec(0,1,0), color=vec(0,1,0))
arrow(pos=vec(0,0,0), axis=vec(0,0,1), color=vec(0,0,1))

# give ball height 5 for test scenarios
ball = Ball(pos = [0.2, 3, 0.4], make_trail=True)
moveables = [ball]
collideables = []

paddle = PingPongPaddle(pos=[0.2,1,0.4])
randVec = (np.random.rand(3)*2)-1
paddle.move(axis=randVec/np.linalg.norm(randVec)*np.array([1, 0.2, 1]), alignCenter=True)
# paddle.move(axis=[1, 0.2, 0], alignCenter=True)
# paddle.vel = np.array([0, 0.1, 0])
# paddle.angularVel = np.array([0, 0.3, 0.3])
collideables.append(paddle)
moveables.append(paddle)


moveablesStates = []
for _ in moveables:
    moveablesStates.append([])
t = 0
frames = 0
simLengthSeconds = 4/speedMultiplier
# simLengthTicks = simLengthSeconds/speedMultiplier
startTime = time.time()

while t < simLengthSeconds:
    moveCam = False
    for _ in range(resolutionMultiplier):
        # startTime = time.time()
        ball.update(collideables=collideables)
        
        if paddle.hitBall:
            moveCam = True
            
        paddle.update()
        t += dt
        frames += 1
        # endTime = time.time()
    if preSimulateThenPlayback:
        for i in range(len(moveables)):
            moveablesStates[i].append(np.copy(moveables[i].pos))
    if not preSimulateThenPlayback:
        for moveable in moveables:
            moveable.show()
        if moveCam and False:
            scene.camera.pos = paddle.paddle.visual.pos + vec(0, 5, 5)
            scene.camera.axis = paddle.paddle.visual.pos - scene.camera.pos
        rate(fps)
endTime = time.time()

if preSimulateThenPlayback:
    print(f"Simulation took: {endTime - startTime} seconds")
    for t in range(len(moveablesStates[0])):
        for i in range(len(moveablesStates)):
            moveables[i].move(pos=moveablesStates[i][t])
            moveables[i].show()
        rate(fps)
else:
    print(f"Simulation was expected to take: {t} seconds and it took: {endTime - startTime} seconds")

<IPython.core.display.Javascript object>

CONTROL
TARGETVEL [-3.50294001  3.52280646  0.37359963] and UNIT [-0.70311913  0.70710678  0.07498988]
ATTEMPTING NUMERICAL SOLUTION
NUMERICAL SOLUTION TOOK 28 ATTEMPTS
FINAL SOLUTION HAS ERROR 0.004389965377182768
Simulation was expected to take: 4.009500000000497 seconds and it took: 13.297249794006348 seconds


#### Scenario 4
This scenario requires some additional imports that we didn't use for the class mainly. They both exist within conda though, so if you want to try it out it shouldn't be too hard
The result should be the robot arm tracking the green sphere as it moves across the space.

In [10]:
from vpython import *
import ikpy.chain
import ikpy.utils.plot as plot_utils
import xml.etree.ElementTree as ET
import numpy as np
import math
my_chain = ikpy.chain.Chain.from_urdf_file("arm_urdf.urdf",active_links_mask=[False, True, True, True, True, True])

def axisAngleRotationMatrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    a = cos(theta / 2.0)
    b, c, d = -axis * sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
class Link:
    def __init__(self, pos=np.zeros(3), size=np.ones(3), axis=None, jointOrigin=None):
        self.pos = pos
        temp = self.pos[2]
        self.pos[2] = self.pos[1]
        self.pos[1] = temp
        self.size = size
        temp = self.size[2]
        self.size[2] = self.size[1]
        self.size[1] = temp
        if np.any(axis == None):
            self.jointAxis = np.array([0., 1., 0.])
        else:
            self.jointAxis = axis
            temp = self.jointAxis[2]
            self.jointAxis[2] = self.jointAxis[1]
            self.jointAxis[1] = temp
        if np.any(jointOrigin == None):
            self.jointOrigin = self.pos.copy()
        else:
            self.jointOrigin = jointOrigin
            temp = self.jointOrigin[2]
            self.jointOrigin[2] = self.jointOrigin[1]
            self.jointOrigin[1] = temp
        self.angle = 0
        self.axis = np.array([1., 0., 0.])
        self.up = np.array([0., 1., 0.])
        self.visual = box(pos=vec(pos[0], pos[1], pos[2]), size=vec(size[0], size[1], size[2]), color=vec(0.5, 0.5, 1))
    def update(self, pos, angle):
        self.pos = pos
        self.visual.pos = vec(self.pos[0], self.pos[1], self.pos[2])
        self.axis = np.dot(axisAngleRotationMatrix(self.jointAxis, angle - self.angle), self.axis)
        self.up = np.dot(axisAngleRotationMatrix(self.jointAxis, angle - self.angle), self.up)
        self.visual.up = vec(*self.up)
        self.visual.axis = vec(*self.axis)
        self.visual.size = vec(*self.size)
        self.angle = angle
        
class Robot:
    def __init__(self, links):
        self.links = links
    def setAngle(self, linkIdx, angle):
        if linkIdx == 0:
            print("BASE LINK CANT MOVE")
            return
        # link = self.links[linkIdx]
        # dAngle = angle - link.angle
        # link.angle = angle
        # rotMat = axisAngleRotationMatrix(link.jointAxis, dAngle)
        # link.axis = np.dot(rotMat, link.axis)
        # prevEndPoint = link.jointOrigin + np.max(link.size)*link.up
        # link.up = np.dot(rotMat, link.up)
        # endPoint = link.jointOrigin + np.max(link.size)*link.up
        # self.links[linkIdx+1].jointOrigin += (endPoint-prevEndPoint)
        # link.pos = link.jointOrigin + np.max(link.size)*link.up*0.5
        # link.visual.pos = vec(*link.pos)
        # link.visual.axis = vec(*link.axis)
        # link.visual.up = vec(*link.up)
        # link.visual.size = vec(*link.size)

        for i in range(linkIdx, len(self.links)):
            
            link = self.links[i]
            if i == 1:
                moveAxis = link.axis
            else:
                moveAxis = link.up
            if i == linkIdx:
                dAngle = angle - link.angle
                link.angle = angle
                rotMat = axisAngleRotationMatrix(link.jointAxis, dAngle)
                prevEndPoint = link.jointOrigin + np.max(link.size)*moveAxis
            link.jointAxis = np.dot(rotMat, link.jointAxis)
            link.axis = np.dot(rotMat, link.axis)
            link.up = np.dot(rotMat, link.up)
            if i == 1:
                moveAxis = link.axis
            else:
                moveAxis = link.up
            endPoint = link.jointOrigin + np.max(link.size)*moveAxis
            if i < len(self.links)-1:
                temp = self.links[i+1].jointOrigin + np.max(self.links[i+1].size)*self.links[i+1].up
                # self.links[i+1].jointOrigin += (endPoint-prevEndPoint)
                # EXTREMELY HARD CODED DONT DO THIS LOL
                
                if i == 2:
                    self.links[i+1].jointOrigin = endPoint-0.07*link.axis
                elif i == 3:
                    self.links[i+1].jointOrigin = endPoint+0.07*link.axis
                elif i == 1:
                    self.links[i+1].jointOrigin += (endPoint-prevEndPoint)
                elif i == 4:
                    self.links[i+1].jointOrigin = endPoint-0.01*link.up
                # elif i == 4:
                #     self.links[i+1].jointOrigin = endPoint-0.07*link.axis
                prevEndPoint = temp
            if i == 5:
                link.pos = link.jointOrigin
            else:
                link.pos = link.jointOrigin + np.max(link.size)*moveAxis*0.5
            link.visual.pos = vec(*link.pos)
            link.visual.axis = vec(*link.axis)
            link.visual.up = vec(*link.up)
            link.visual.size = vec(*link.size)

scene = canvas()


# Parse the URDF file
tree = ET.parse('arm_urdf.urdf')
root = tree.getroot()

# Access robot properties
robot_name = root.attrib.get('name')
# print(robot_name)
summedPos = np.zeros(3)
joints = {}
for joint in root.findall('joint'):
    parent = joint.findall('parent')[0].attrib['link']
    child = joint.findall('child')[0].attrib['link']
    origin = np.array([float(i) for i in joint.findall('origin')[0].attrib['xyz'].split(" ")])
    axis = np.array([float(i) for i in joint.findall('axis')[0].attrib['xyz'].split(" ")])
    # print("JOINT", parent, child, origin, axis)
    joints[child] = [origin, axis]

links = []
for link in root.findall('link'):
    name = link.attrib['name']
    # print(name)
    visual = link.findall('visual')[0]
    origin = visual.findall('origin')[0]
    geom = visual.findall('geometry')[0]
    if len(geom.findall('box'))>0:
        # print("BOX")
        geom = geom.findall('box')[0]
        size = np.array([float(i) for i in geom.attrib['size'].split(" ")])
    else:
        # print("CYLINDER")
        geom = geom.findall('cylinder')[0]
        radius = float(geom.attrib['radius'])
        length = float(geom.attrib['length'])
        size = np.array([radius*2, radius*2, length])
    origin = np.array([float(i) for i in origin.attrib['xyz'].split(" ")])
    
    axis = None
    jointOrigin = None
    if name in joints:
        jointOrigin = joints[name][0]
        axis = joints[name][1]
        summedPos += jointOrigin
    origin += summedPos
    
    # print("Pos", origin)
    # print("Size", size)
    links.append(Link(origin, size, axis, summedPos.copy()))
# links[3].visual.color=vec(1, 0, 0)
robot = Robot(links)
# robot.setAngle(1, 3.14159/4)
# robot.setAngle(2, 3.14159/4)
# robot.setAngle(3, 3.14159/4)
# robot.setAngle(4, 3.14159/4)
# robot.setAngle(5, 3.14159/4)

dt = 0.01
time = np.arange(0, 4, dt)
xposes = np.linspace(-.5, .3, len(time))
yposes = np.linspace(-.3, .5, len(time))
zposes = np.linspace(0.5, 0.7, len(time))
target_position = [ 0.3048, 0.3048,0.1]
ik = my_chain.inverse_kinematics(target_position)
endGoal = sphere(pos=vec(0, 0, 0), radius=0.05, color=vec(0, 1, 0))
arrow(pos=vec(0,0,0), axis=vec(0.2,0,0), color=vec(1,0,0))
arrow(pos=vec(0,0,0), axis=vec(0,0,0.2), color=vec(0,0,1))


for i, t in enumerate(time):

    
    target_position = [xposes[i], yposes[i], zposes[i]]
    endGoal.pos = vec(target_position[0], target_position[2], -target_position[1])
    target_orientation = [0, 0, 1]

    ik = my_chain.inverse_kinematics(target_position, target_orientation, initial_position=ik.copy(), orientation_mode="Z")
    # ik = my_chain.inverse_kinematics(target_position, initial_position=ik.copy())
    computed_position = my_chain.forward_kinematics(ik)
    angles = ik.tolist()
    # angles[1] += 3.14159
    for i in range(1, len(angles)):
        robot.setAngle(i, angles[i])
    rate(12)

ValueError: Your active links mask length of 6 is different from the number of your links, which is 9

# Evaluation
## How my results relate
My results are extremely relevant to my justification! In general my simulation worked really well and is addressing a lot of my problems/questions that inspired me to make it. The only limiting factor is having more time to add more features.

In the results you saw that the simulation can accurately simulation the motion of a ping pong ball colliding with arbitratrily positioned and rotated objects. That we can control a ping pong paddle to move to any desired postiion. And that we have setup a model of a robot arm which can track a target position.

These results mean that all of my assumptions about how this system would work are correct, and I am actually not too far from getting to my desired goal of getting the robot to playing ping pong. Without this simulation, it would have been much harder and more dangerous (danger in terms of breaking expensive hardware) to get to where I am right now!

## What are limitations of the model
### Spin
The model currently doesn't account for ball spin, which in real life will effect the trajectory a notable amount
### Air disturbances/wind
The model assumes the air is perfectly still, and doesn't includ any noise that would happen in real life.
### Robot control errors
In real life we wouldn't be able to control the robot to move to exactly the right location with the exact right velocity all the time. There would be small errors which would effect the balls trajectory.

## Given the limitations, how can I overcome them and continue to explore
All three limitations I just referenced prevent the ball from moving in the exact ways that the simulation expects. But the simulation is using open loop control on the ball, its assuming it will move perfectly as expected. To overcome this, we can use a sensor to track the location of the ball and close the loop. All the paddle velocity and trajectory planning concepts that I mentioned, can be recomputed several times throughout the balls trajectory in the air, insuring that any errors that are introduced are compensated for!

I can also continue to explore by simulating these limitations. Add random noise to the air, and robot movements. And also simulate ball spin!

## Challenges
I encountered a lot of challenges in terms of this being a complicated problem. But I was able to overcome them through many hours of debugging and research. Right now the project is at a point where there is no feature that I haven't been able to accomplish yet!

Another challenge with this project was the limitation of vpython and jupyter notebooks being very slow. In the future I would plan to overcome these challenges by either further optimizing my code, and/or switching from vpython/jupyter to pyopengl (or another more efficient 3d rendering program) and just individual python files.

## Given approach and relevant course concepts, what alternative approches could be valid?
Other valid approaches could include using reinforcement learning based control, like PPO which I mentioned earlier, instead of fully needing to calculate all the equations for finding optimal paddle velocities etc.

In terms of the physics part, I think there is really no good replacement for this method. Trajectories over time can't really be done analytically when there are collisions/discontinuities involved, so the numerical integration technique really is the best solution for simulating the physics. And simulating the physics is required for testing the control simulations, whether its PPO or my analytical trajectory calculating paddle velocity finding stuff.

If I weren't to use simulation at all, I would struggle greatly with debugging, as there are so many things that need to be done correctly before the robot arm can do anything remotely useful.
I could take alternative routes like using the xbox kinect/3d camera to track the ball and paddle as I swing it with my own arms, and write prediction code based on collision parameters, and then compare it to what the kinect tracks. But that would be much more difficult that just this simulation that I did

# Conclusions and Future Work
All in all this went really well and I am really happy with the result! The simulation is able to accurately determine the paddle velocity needed to hit the ball to a desired location, and then figure out the robot arm joint angles needed to move to a certain position. The only step missing that I haven't had time to implement, is connecting those features. And making it so that the desired paddle velocity would create a robot arm trajectory to follow, which would be used for computing the joint angles.

I plan on continuing with this project over the next week before I go home for the summer. Hopefully I will get the robot arm fully working within the simulation, and maybe partially working in real life (although I'd have to work super efficiently to get that done on time).

Other features I could include would be simulating ball rolling, spin, noise, different types of control, different geometrical shapes for the ball to bounce on, and much more!

# References

[Physics Libre Text on CoM Collisions](https://phys.libretexts.org/Bookshelves/University_Physics/Mechanics_and_Relativity_(Idema)/04%3A_Momentum/4.08%3A_Elastic_Collisions_in_the_COM_Frame)

[Youtube video on rigid body dynamics](https://www.youtube.com/watch?v=4r_EvmPKOvY&list=PLwMZtAEBQ8ZywWPf6twbspmYzGg0Fr2DJ&index=5)

[Thread on inverse cross product where I got idea on how to solve for angular velocity given delta-rotation](https://math.stackexchange.com/questions/32600/whats-the-opposite-of-a-cross-product)

[resolved motion rate control](https://roboticsknowledgebase.com/wiki/planning/resolved-rates/)

[Proximal Policy Optimization](https://en.wikipedia.org/wiki/Proximal_policy_optimization)

[Physion Physics Engine](https://physion.net/)

[PhET Collision Lab](https://phet.colorado.edu/sims/html/collision-lab/latest/collision-lab_all.html)

[PhET Projectile Motion Lab](https://phet.colorado.edu/sims/html/projectile-motion/latest/projectile-motion_all.html)